## Optimization Program for Venue Series
#### Copyright © 2024 Hoshino Math Services

In [1]:
# Import the Python Modules.

import time
import math
import numpy as np
import pandas as pd
import csv
from ortools.linear_solver import pywraplp

AttendeeFile = pd.read_csv('Attendee Names.csv')
AttendeeList = []
for i in range(len(AttendeeFile)):
    if AttendeeFile["Checked-in"][i] != '0':
        AttendeeName = AttendeeFile["First name"][i] + " " + AttendeeFile["Last name"][i]
        if AttendeeName in AttendeeList:
            print("ERROR", AttendeeName, "appears twice")
        else:
            AttendeeList.append(AttendeeName)
                     
VendorFile = pd.read_csv('Vendor Names.csv')
VendorList = []
for i in range(len(VendorFile)):
    if VendorFile["Present"][i] != '0':
        VendorName = VendorFile["Vendor Name"][i]
        if VendorName in VendorList:
            print("ERROR", VendorName, "appears twice")
        else:
            VendorList.append(VendorName)

VRatingFile = pd.read_csv('Vendor Ratings.csv')
VRInfo = VRatingFile.values.tolist()

ARatingFile = pd.read_csv('Attendee Ratings.csv')
ARInfo = ARatingFile.values.tolist()

V = len(VendorList)
A = len(AttendeeList)

print("There are", V, "vendors and", A, "attendees that are present")

There are 21 vendors and 30 attendees that are present


In [2]:
#for i in range(len(VRInfo)):
#    AttendeeName = VRInfo[i][0] + " " + VRInfo[i][1]
#    if not AttendeeName in AttendeeList:
#        print(AttendeeName, "does not appear in the list of Attendees")

In [3]:
# Determine the score of Vendor V being matched with Attendee A

VAScore = [[0 for a in range(A)] for v in range(V)]
VAScore1 = [[0 for a in range(A)] for v in range(V)]
VAScore2 = [[0 for a in range(A)] for v in range(V)]


# Assign 3 points for each occurrence of an Attendee picking a Vendor
# Assign 2 points for each occurrence of a Vendor picking an Attendee

for i in range(len(ARatingFile)):
    AttendeeName = ARatingFile["First Name"][i] + " " + ARatingFile["Last Name"][i]
    if AttendeeName in AttendeeList:
        AttendeeIndex = AttendeeList.index(AttendeeName)
        for VendorName in VendorList:
            VendorIndex = VendorList.index(VendorName)
            Response = ARatingFile[VendorName][i]
            if Response == 'Very Interested.': Score = 2
            elif Response == 'Not Interested.': Score = 0
            elif Response == 'Interested.': Score = 1
            else: print("ERROR", Response)
            VAScore1[VendorIndex][AttendeeIndex] = Score
            
for i in range(len(VRatingFile)):
    AttendeeName = VRatingFile["FIRST"][i] + " " + VRatingFile["LAST"][i]
    if AttendeeName in AttendeeList:
        AttendeeIndex = AttendeeList.index(AttendeeName)
        for VendorName in VendorList:
            VendorIndex = VendorList.index(VendorName)
            Score = VRatingFile[VendorName][i]
            VAScore2[VendorIndex][AttendeeIndex] = Score
            
for v in range(V):
    for a in range(A):
        VAScore[v][a] = 3*VAScore1[v][a] + 2*VAScore2[v][a]

        
# Assign -100 points for each occurrence of (Vendor, Attendee) pair where neither side has picked the other

for v in range(V):
    for a in range(A):
        if VAScore[v][a] == 0:
            VAScore[v][a] = -100
            
        
        
# Assign -10,000 points for each occurrence of (Vendor, Attendee) pair where the Vendor is not Local 
# and the Attendee is not National, they cannot be paired   

NonLocalVendorList = []
for i in range(len(VendorFile)):
    VendorName = VendorFile["Vendor Name"][i]
    if VendorName in VendorList:
        if VendorFile["Local"][i] != 'x':
            NonLocalVendorList.append(VendorName)
                
NonNationalAttendeeList = []
for i in range(len(AttendeeFile)):
    AttendeeName = AttendeeFile["First name"][i] + " " + AttendeeFile["Last name"][i]
    if AttendeeName in AttendeeList:
        if not 'ational' in AttendeeFile["Where do your events take place?"][i]:
            NonNationalAttendeeList.append(AttendeeName)
        
for v in range(V):
    for a in range(A):
        if VendorList[v] in NonLocalVendorList and AttendeeList[a] in NonNationalAttendeeList:
            VAScore[v][a] =- 10000
        
        
# Determine the number of meeting rounds

R = 12

In [4]:
# Optimize the assignment of Attendees to Vendors

StartTime = time.time()

solver = pywraplp.Solver("Venue Series", pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

Vendors = range(len(VendorList))
Attendees = range(len(AttendeeList))
Rounds = range(12)

x = {}
for v in Vendors:
    for a in Attendees:
        for r in Rounds:
            x[v,a,r] = solver.IntVar(0,1, "x[%d,%d,%d]" % (v,a,r))

        
# CONSTRAINT 1: Each Vendor has one Attendee Meeting per round
for v in Vendors:
    for r in Rounds:
        solver.Add(sum(x[v,a,r] for a in Attendees) == 1)
        
        
# CONSTRAINT 2: Each Attendee has at most one Vendor Meeting per round
for a in Attendees:
    for r in Rounds:
        solver.Add(sum(x[v,a,r] for v in Vendors) <= 1)
        
        
# CONSTRAINT 3: Each Attendee has approximately the same number of Vendor Meetings
for a in Attendees:
    solver.Add(sum(x[v,a,r] for v in Vendors for r in Rounds) >= math.floor(V*12/A))
    solver.Add(sum(x[v,a,r] for v in Vendors for r in Rounds) <= math.ceil(V*12/A))
    
    
# CONSTRAINT 4: Each (Vendor, Attendee) pair can only meet once
for v in Vendors:
    for a in Attendees:
        solver.Add(sum(x[v,a,r] for r in Rounds) <= 1)
    
    
# Run the optimization
solver.Maximize(solver.Sum(VAScore[v][a] * x[v,a,r] for v in Vendors for a in Attendees for r in Rounds))
sol = solver.Solve()

Solution = []
for v in Vendors:
    for a in Attendees:
        for r in Rounds:
            if x[v,a,r].solution_value()==1:
                Solution.append([v,a,r])

In [5]:
# Export to Excel

OurColumns = ["Attendee ID", "Attendee Name", "Attendee Email", "Attendee Company"]
for r in Rounds:
    OurColumns.append("Round " + str(r+1))

M = []
for i in range(len(AttendeeFile)):
    AllVendors = ["No Meeting Scheduled" for r in Rounds]
    AttendeeName = AttendeeFile["First name"][i] + " " + AttendeeFile["Last name"][i]
    AttendeeEmail = AttendeeFile["Email"][i]
    AttendeeCompany = AttendeeFile["Company/Organization"][i]
    if AttendeeName in AttendeeList:
        AttendeeID = AttendeeList.index(AttendeeName)
        for mypair in Solution:
            if mypair[1] == AttendeeID:
                AllVendors[mypair[2]] = "V"+str(mypair[0])
                #AllVendors[mypair[2]] = VendorList[mypair[0]]
        M += [["A"+str(AttendeeID), AttendeeName, AttendeeEmail, AttendeeCompany] + AllVendors[0:R]]

FinalMatrix = pd.DataFrame(M, columns=OurColumns)
FinalMatrix.to_csv("Output File - Attendee Assignments.csv", index=False)

    
OurColumns = ["Vendor ID", "Vendor Rep", "Vendor Email", "Vendor Name"]
for r in Rounds:
    OurColumns.append("Round " + str(r+1))

M = []
for i in range(len(VendorFile)):
    AllAttendees = ["No Meeting Scheduled" for r in Rounds]
    VendorName = VendorFile["Vendor Name"][i]
    VendorEmail = VendorFile["Email Address"][i]
    VendorRep = VendorFile["Vendor Rep"][i]
    
    if VendorName in VendorList:
        VendorID = VendorList.index(VendorName)
        for mypair in Solution:
            if mypair[0] == VendorID:
                AllAttendees[mypair[2]] = "A"+str(mypair[1])
                #AllAttendees[mypair[2]] = AttendeeList[mypair[1]]
                
    M += [["V"+str(VendorID), VendorRep, VendorEmail, VendorName] + AllAttendees[0:R]]
    
FinalMatrix = pd.DataFrame(M, columns=OurColumns)
FinalMatrix.to_csv("Output File - Vendor Assignments.csv", index=False)

In [6]:
# Output the final score with the total running time

TotalScore = 0
for temp in Solution:
    TotalScore += VAScore[temp[0]][temp[1]]
      
TotalTime = time.time() - StartTime
print("Optimization Complete in", round(TotalTime,4), "seconds")   
print("The total score is", TotalScore, "points")
print("")

# Output any matches where neither the vendor nor the attendee has selected the other

for temp in Solution:
    if VAScore[temp[0]][temp[1]] == -100:
        print("Vendor", VendorList[temp[0]], "and Attendee", AttendeeList[temp[1]], "did not select each other")
        
        
# Output any matches where neither the vendor is not Local and the attendee is not National

for temp in Solution:
    if VAScore[temp[0]][temp[1]] < -9000:
        print("Vendor", VendorList[temp[0]], "is not local and Attendee", AttendeeList[temp[1]],
              "is not national")

Optimization Complete in 11.0559 seconds
The total score is 1500 points

Vendor Delta Hotels by Marriott Calgary Downtown and Attendee Stephanie Lewis did not select each other
Vendor Discover Saskatoon and Attendee Susan Ramsay did not select each other
Vendor Discover Saskatoon and Attendee Hazel Thalakkat did not select each other
